<a href="https://colab.research.google.com/github/pavass/spam-email-predictor/blob/master/Email_Spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

In [0]:
from google.colab import files
uploaded=files.upload()


Saving emails.csv to emails.csv


In [0]:
df=pd.read_csv('emails.csv')
df.head(5)

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [0]:
df.shape

(5728, 2)

In [0]:
df.columns

Index(['text', 'spam'], dtype='object')

In [0]:
#check for duplicates
df.drop_duplicates(inplace = True)

In [0]:
df.shape

(5695, 2)

In [0]:
#show missing data from each column(NAN,NaN,etc)
df.isnull().sum()

text    0
spam    0
dtype: int64

In [0]:
#download stopwords package
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
def process_text(text):
  #1 remove punctuation
  #2 remove stopwords
  #3 return list of clean text words

  #1
  nopunc=[char for char in text if char not in string.punctuation]
  nopunc=''.join(nopunc)

  #2
  clean_words=[word for word in nopunc.split() if word.lower() not in stopwords.words('english')]


  #3
  return clean_words


In [0]:
#show tokenization
df['text'].head().apply(process_text)

0    [Subject, naturally, irresistible, corporate, ...
1    [Subject, stock, trading, gunslinger, fanny, m...
2    [Subject, unbelievable, new, homes, made, easy...
3    [Subject, 4, color, printing, special, request...
4    [Subject, money, get, software, cds, software,...
Name: text, dtype: object

In [0]:
#example
message4='hello world hello hello world play'
message5='test test test test one hello'
print(message4)
print()

#convert text to a matrix of token counts
from sklearn.feature_extraction.text import CountVectorizer
bow4=CountVectorizer(analyzer=process_text).fit_transform([[message4],[message5]])
print(bow4)
print(bow4.shape)

hello world hello hello world play

  (0, 0)	3
  (0, 4)	2
  (0, 2)	1
  (1, 0)	1
  (1, 3)	4
  (1, 1)	1
(2, 5)


In [0]:
#convert collection of text to matrix  of tokens
from sklearn.feature_extraction.text import CountVectorizer
messages_bow=CountVectorizer(analyzer=process_text).fit_transform(df['text'])


In [0]:
#split data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(messages_bow,df['spam'],test_size=0.20,random_state=0)

In [0]:
#get shape of messages_bow
messages_bow.shape

(5695, 37229)

In [0]:
#create and train naive bayes classifier(multinomial)
from sklearn.naive_bayes import MultinomialNB
classifier=MultinomialNB().fit(x_train,y_train)

In [0]:
#print predictions
print(classifier.predict(x_train))

#print actual values
print(y_train.values)

[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [0]:
#evaluate model on training data set
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
pred=classifier.predict(x_train)
print(classification_report(y_train,pred))
print()
print('Confusion Matrix: \n',confusion_matrix(y_train,pred))
print('Accuracy:',accuracy_score(y_train,pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3457
           1       0.99      1.00      0.99      1099

    accuracy                           1.00      4556
   macro avg       0.99      1.00      1.00      4556
weighted avg       1.00      1.00      1.00      4556


Confusion Matrix: 
 [[3445   12]
 [   1 1098]]
Accuracy: 0.9971466198419666


In [0]:
 #print predictions
print(classifier.predict(x_test))

#print actual values
print(y_test.values)

[1 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]


In [0]:
#evaluate model on training data set
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
pred=classifier.predict(x_test)
print(classification_report(y_test,pred))
print()
print('Confusion Matrix: \n',confusion_matrix(y_test,pred))
print('Accuracy:',accuracy_score(y_test,pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       870
           1       0.97      1.00      0.98       269

    accuracy                           0.99      1139
   macro avg       0.98      0.99      0.99      1139
weighted avg       0.99      0.99      0.99      1139


Confusion Matrix: 
 [[862   8]
 [  1 268]]
Accuracy: 0.9920983318700615
